<a href="https://colab.research.google.com/github/xx529/Algorithm/blob/main/Tobit%20Model%20-%20%20Censored%20Normal%20Regression/Tobit%20Model%20-%20%20Censored%20Normal%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tobit Model -  Censored Normal Regression

**关于算法的故事**

在任一给定年份，有相当数量家庭的医疗保险费用支出为0

因此，虽然年度家庭医疗保险费用支出的总体分布散布于一个很大的正数范围内，但在数字0上却相当集中

# Algorithm

**定义**

1. 因变量 $y$ 在正值上大致是连续分布，但也包含一部分以正概率取值为0的观测，对于 $y$ 如下定义，$y^*$ 是一个隐藏的变量，它与0共同决定了观测 $y$

$$y=\left\{
\begin{aligned}
y^* ,& y^* \geqslant 0 \\
- ,& y^* < 0 
\end{aligned}
\right.
$$

2. 隐藏变量 $y^*$ 由一个参数为 $\beta$ 的线性方程和 $\epsilon$，定义如下
2
$$y^* = x^T\beta + \epsilon, \text{ 其中 } \epsilon \sim N(0, \sigma^2)$$
$$$$

3. 需要优化的参数就是 $\beta$ 和 $\sigma$

$$\hat{\theta}= (\hat{\beta}, \hat{\sigma}^2)$$

**目标函数**

1. 定义一个 indicator variable $d$，取 $L=0$ 如下:

$$d=\left\{
\begin{aligned}
1 ,& y > L\\
0 ,& y = L 
\end{aligned}
\right.$$

2. 由定义式可知 $y^* \sim (x^T\beta, \sigma^2)$

$$y^* = x^T\beta + \epsilon \rightarrow y^* \sim (x^T\beta, \sigma^2)$$

3. 当 $y^*<L$，用 CDF概率分布函数，当 $y^*>L$ 可观测时候用 PDF概率密度函数，这里取 $L=0$

$$f(y)=\left\{
\begin{aligned} F^*(0)=1-\Phi(\frac{x^T\beta}{\sigma}), d=0\\
\frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{1}{2\sigma^2}(y-x^T\beta)^2), d=1
\end{aligned}
\right.
$$

4. 综合上面两个分段函数就有如下形式，也就是我们通过 MLE 最大化的函数

$$f(y) =[\frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{1}{2\sigma^2}(y-x^T\beta)^2)]^d[1-\Phi(\frac{x^T\beta}{\sigma})]^{1-d}$$

**参数梯度**

对以上公式取 $log$ 之后的参数梯度，其中 $\phi_i=\phi(\frac{x_i^T\beta}{\sigma})$，$\Phi_i=\Phi(\frac{x_i^T\beta}{\sigma})$

$$\frac{\partial ln L_N}{\partial \beta}=\sum^N_{i=1}\frac{1}{\sigma^2}(d_i(y_i-x^T\beta)-(1-d_i)\frac{\sigma\phi_i}{1-\Phi_i})x_i$$


$$\frac{\partial ln L_N}{\partial \sigma^2}=\sum^N_{i=1}[d_i(-\frac{1}{2\sigma^2}+\frac{(y_i-x^T\beta)^2}{2\sigma^4})+(1-d_i)\frac{\phi_ix_i^T\beta}{1-\Phi_i} · \frac{1}{2\sigma^3}]$$

**优化方法**

[Proximal Methods](https://github.com/xx529/Algorithm/blob/main/Proximal%20Algorithms%20-%20L1%20Regularization/Proximal%20Algorithms%20-%20L1%20Regularization.ipynb)

In [212]:
import jax
import jax.numpy as jnp
from jax import grad, vmap, jit
from jax.lax import cond
import numpy as np

# Experiment Data

In [242]:
key = jax.random.PRNGKey(529)
batch = 10000
input_dim = 4
L = 0

true_beta = jnp.array([2.0, 3.0, -2.0, 0.0])
true_sigma = 2.0
epsilon = jax.random.normal(key, (batch, )) * (true_sigma**2)

x = jax.random.normal(key, (batch, input_dim))
y_star = jnp.dot(x, true_beta) + epsilon
d = (y_star > L).astype(jnp.float32)
y =  d * y_star

beta = jax.random.normal(key, (input_dim, ))
sigma = jax.random.normal(key)

print('epsilon mean', epsilon.mean())
print('epsilon std', epsilon.std())
print('y_star std', y_star.std())
print('x_dot_b mean', jnp.dot(x, true_beta).mean())
print('y_star mean', y_star.mean())

epsilon mean -0.011463349
epsilon std 4.009254
y_star std 5.6949024
x_dot_b mean -0.0027797106
y_star mean -0.014243055


In [214]:
y

DeviceArray([-0.        ,  1.8103466 , -0.        , ..., -0.        ,
              0.15429918, -0.        ], dtype=float32)

In [215]:
y_star

DeviceArray([ -1.7571797 ,   1.8103466 ,  -9.282396  , ...,  -6.6482935 ,
               0.15429918, -10.751861  ], dtype=float32)

# Implementation

### General Version

In [216]:
def normal_cdf(x):
    return jax.scipy.stats.norm.cdf(x, loc=0, scale=1)


def normal_pdf(x):
    return jax.scipy.stats.norm.pdf(x, loc=0, scale=1)


def linear(x, beta):
    return jnp.dot(x, beta)


def log_mle(x, beta, sigma, y_true, l):
    d = (y_true > l).astype(jnp.float32)
    temp1 = d * jnp.log(1 / (jnp.sqrt(2 * jnp.pi * sigma**2))) + (-1 / (2 * sigma**2)) * (y_true - linear(x, beta))**2
    temp2 = (1 - d) * jnp.log(1 - normal_cdf(linear(x, beta) / sigma) + 0.01)
    return jnp.mean(temp1 + temp2)


def tobit_model_grad(x, y_true, beta, sigma):
    x_dot_beta = linear(x, beta)
    residual = y_true - x_dot_beta
    sigma_square = sigma**2
    cdf = normal_cdf(x_dot_beta / sigma)
    pdf = normal_pdf(x_dot_beta / sigma)
    d = (y_true > L).astype(jnp.float32)
    
    d_beta_temp = d*residual - (1-d) * sigma * pdf / (1 - cdf + 0.01)
    d_beta = - (1 / sigma_square) * jnp.dot(d_beta_temp, x) / x.shape[0]
    
    d_sigma2_temp_1 = d * (-1 / (2 * sigma_square) + (residual**2) / (2 * sigma**4))    
    d_sigma2_temp_2 = 1 / (2 * sigma**3 ) * (1 - d) * (pdf * x_dot_beta) / (1 - cdf + 0.01)
    d_sigma2 = - jnp.mean(d_sigma2_temp_1 + d_sigma2_temp_2)
    
    return d_beta, d_sigma2


def tobit_model_train(x, y_true, beta, sigma, lr, l, esp, max_iter):
    current_mle, current_esp, current_iter = jnp.inf, 1, 1

    while current_iter <= max_iter and current_esp >= esp:
        d_beta, d_sigma2 = jit(tobit_model_grad)(x, y, beta, sigma)

        beta = beta - lr * d_beta
        sigma = jnp.sqrt(sigma**2 - lr * d_sigma2)

        mle = log_mle(x, beta, sigma, y_true, l)
        current_esp = jnp.abs(mle - current_mle)
        current_mle = mle
        current_iter += 1

    return beta, sigma

### JAX Version
1. Proximal Method
2. Accelerated by jit

In [217]:
def normal_cdf(x):
    return jax.scipy.stats.norm.cdf(x, loc=0, scale=1)


def normal_pdf(x):
    return jax.scipy.stats.norm.pdf(x, loc=0, scale=1)


def linear(x, beta):
    return jnp.dot(x, beta)


def log_mle(x, beta, sigma, y_true, l):
    d = (y_true > l).astype(jnp.float32)
    temp1 = d * jnp.log(1 / (jnp.sqrt(2 * jnp.pi * sigma**2))) + (-1 / (2 * sigma**2)) * (y_true - linear(x, beta))**2
    temp2 = (1 - d) * jnp.log(1 - normal_cdf(linear(x, beta) / sigma) + 0.01)
    return jnp.mean(temp1 + temp2)


@jax.partial(vmap, in_axes=(0, 0, None, None, None))
def select_grad(x, y_true, beta, sigma, L):
    return jax.lax.cond(
        y_true > L,
        lambda _: ( # > L
            - 1 / (sigma**2) * jnp.dot((y_true - linear(x, beta)), x), # beta
            - (- 1 / (2*sigma**2) + (y_true - linear(x, beta))**2 / (2 * sigma**4)) # sigma
        ), 
        lambda _: ( # < L
             1 / (sigma**2) * jnp.dot((sigma * normal_pdf(linear(x, beta) / sigma) / (1 - normal_cdf(linear(x, beta) / sigma) + 0.01)), x), # beta
            - 1 / (2 * sigma**3) * normal_pdf(linear(x, beta) / sigma) * linear(x, beta) / (1 - normal_cdf(linear(x, beta) / sigma) + 0.01) # sigma
        ), 
        operand = None
    )


def jax_soft_threshold(w, threshold):
    return jax.lax.cond(
        w > threshold, 
        lambda _: w - threshold,
        lambda _: jax.lax.cond(
            w < - threshold,
            lambda _: w + threshold,
            lambda _: 0.0,
            None
            ),
        None
        )
    

def jax_proximal_method_update(beta, sigma, x, y_true, l, lr, penalty):
    d_beta, d_sigma2 = select_grad(x, y_true, beta, sigma, l)
    w_beta = beta - lr * jnp.mean(d_beta, axis=0)
    w_sigma = jnp.sqrt(sigma**2 - lr * jnp.mean(d_sigma2))

    return vmap(jax_soft_threshold, in_axes=(0, None))(w_beta, lr * penalty), jax_soft_threshold(w_sigma, lr * penalty)


def cond_fun(val):
    x, y_true, beta, sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp = val
    result = val[7] <= val[8]
    # result = (val[8] > val[9]) 
    return result


def body_fun(val):
    x, y_true, beta, sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp = val

    new_beta, new_sigma = jax_proximal_method_update(beta, sigma, x, y_true, L, lr, penalty)

    new_mle = log_mle(x, new_beta, new_sigma, y_true, L)
    current_esp = jnp.abs(current_mle - new_mle)
    current_iter += 1

    return (x, y_true, new_beta, new_sigma, lr, L, penalty, current_iter, max_iter, new_mle, current_esp, esp)


@jit
def jax_tobit_model_train(init_val):
    return jax.lax.while_loop(cond_fun, body_fun, init_val)

# Code Test



In [235]:
lr = 1e-3
esp = 1e-6
max_iter = 10000
current_iter = 1
current_mle = log_mle(x, beta, sigma, y, L)
last_mle = log_mle(x, beta, sigma, y, L) + 1
current_esp = jnp.abs(current_mle - last_mle) 

In [243]:
print(true_beta)
print(true_sigma)

[ 2.  3. -2.  0.]
2.0


In [237]:
%%time
result = tobit_model_train(x, y, beta, sigma, lr, L, esp, max_iter)
print(result[0])
print(result[1])

[ 1.553281   1.6745833 -1.7175509  0.0360731]
2.4013102
CPU times: user 2min 20s, sys: 28.5 s, total: 2min 49s
Wall time: 1min 57s


In [238]:
%%time
penalty = 0.00

init_val = (x, y, beta, sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)

print(result[2])
print(result[3])

[ 1.553281    1.6745833  -1.7175509   0.03607311]
2.4013102
CPU times: user 1.27 s, sys: 273 ms, total: 1.55 s
Wall time: 1.55 s


In [241]:
%%time
penalty = 0.05

init_val = (x, y, beta, sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)

print(result[2])
print(result[3])

[ 1.3104256  1.6382418 -1.4624276  0.       ]
2.081685
CPU times: user 1.2 s, sys: 335 ms, total: 1.54 s
Wall time: 1.54 s


In [240]:
%%time
penalty = 0.1

init_val = (x, y, beta, sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)

print(result[2])
print(result[3])

[ 1.1199226  1.597426  -1.2620401  0.       ]
1.8571857
CPU times: user 1.27 s, sys: 276 ms, total: 1.55 s
Wall time: 1.55 s




---


# Others
* 检查当 ϵ 为柯西分布时对整个估计的影响
* 对参数估计整体偏小

In [224]:
key = jax.random.PRNGKey(300)
batch = 10000
input_dim = 6
L = 0

true_beta = jnp.array([2.0, 3.0, -2.0, 0.0, 3.0, 5.0])
cauchy_epsilon = jax.random.cauchy(key,(batch, ))

cauchy_x = jax.random.normal(key, (batch, input_dim))
cauchy_y_star = jnp.dot(cauchy_x, true_beta) + cauchy_epsilon
cauchy_d = (cauchy_y_star > L).astype(jnp.float32)
cauchy_y =  d * cauchy_y_star

cauchy_beta = jax.random.normal(key, (input_dim, ))
cauchy_sigma = jax.random.normal(key)

In [225]:
true_beta

DeviceArray([ 2.,  3., -2.,  0.,  3.,  5.], dtype=float32)

In [226]:
lr = 1e-3
esp = 1e-6
max_iter = 10000
current_iter = 1
current_mle = log_mle(x, beta, sigma, y, L)
last_mle = log_mle(x, beta, sigma, y, L) + 1
current_esp = jnp.abs(current_mle - last_mle) 
penalty = 0.0

init_val = (cauchy_x, cauchy_y, cauchy_beta, cauchy_sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)
print(result[2])
print(result[3])

[ 1.4167621  0.5719836 -0.3839866  0.1592816  1.8818617  1.4384959]
3.8122559


In [227]:
lr = 1e-3
esp = 1e-6
max_iter = 20000
current_iter = 1
current_mle = log_mle(x, beta, sigma, y, L)
last_mle = log_mle(x, beta, sigma, y, L) + 1
current_esp = jnp.abs(current_mle - last_mle) 
penalty = 0.00

init_val = (cauchy_x, cauchy_y, cauchy_beta, cauchy_sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)
print(result[2])
print(result[3])

[ 1.2742995   0.78964996 -0.6868022  -0.00281969  1.7733035   1.7916064 ]
4.2732177


In [228]:
lr = 1e-3
esp = 1e-6
max_iter = 30000
current_iter = 1
current_mle = log_mle(x, beta, sigma, y, L)
last_mle = log_mle(x, beta, sigma, y, L) + 1
current_esp = jnp.abs(current_mle - last_mle) 
penalty = 0.00

init_val = (cauchy_x, cauchy_y, cauchy_beta, cauchy_sigma, lr, L, penalty, current_iter, max_iter, current_mle, current_esp, esp)
result = jax_tobit_model_train(init_val)
print(result[2])
print(result[3])

[ 1.1933279   0.92393094 -0.8688001  -0.09837008  1.7164546   2.0070918 ]
4.568091
